# Ранбук по теме Работа с командной строкой Linux p2

### Дисклеймер

*Shell, Terminal, Bash и Консоль &ndash; это не муж и жена, а 4 разных человека, в смысле все эти слова обозначают почти одно и то же, разница в том, что Terminal &ndash; это программа, в которой открывается командная строка линукс, Shell (дословно с английского &ndash; оболочка) &ndash; это собственно командная строка, а Bash &ndash; конкретная имплементация (как ни странно на жаргоне &ndash; оболочка) этой самой командной строки (помимо Bash существуют и другие, имеющие некоторые, как правило не очень сильныке отличия от Bash, напр., Zsh, Ash, Sh и др.)*

## Bash как язык

### Переменные, циклы и subshell

Как в любом языке программирования в bash есть переменные. Они динамически типизируемые, как и в python.

In [ ]:
%%bash
# Этот код работает

a="Hello"
b="world!"
c="$a ${b}"  # конкатенация строк, для удобства можно использовать не $variable_name, а ${variable_name}, если
             # переменная окружена текстом; переменные вызываются, если находятся в двойных кавычках

echo $c

echo '$c'    # в одинарных ковычках переменные не подставляются
echo "\$c"   # ещё один способ экранировать

Можно складывать числа:

In [ ]:
%%bash
# Этот код работает

b=4

echo $((1 + b))

echo $(((b * 5 - 143) / 10 + 2 ** 2))

А ещё, можно присваивать в переменную результат выполнения пайплайна &ndash; это делается через вызов subshell (фактически создаётся новая дочерняя (child) сессия bash в которой выполняется код):

In [ ]:
%%bash
# Этот код работает

current_path="$(pwd)"
a="$(ls -l)"
a_path="$(echo "Bash path from Subshell: $current_path")"  # в subshell копируются все переменные из текущей сессии

echo "Bash:"
ls -l
echo "Subshell:"
echo "$a"

echo "Bash path: $current_path"

echo "$a_path"

Однако, как насчёт изменения переменных в subshell?

In [ ]:
%%bash
# Этот код работает

a=1

echo "Begining: $a"

echo "In Subshell: $(a=3; b=5; echo $((a+b)))"

echo "Variables: a=$a, b=$b"  # как видим, переменные subshell недоступны в основном процессе и изменение переменных
                              # основного процесса в subshell не отражается вне её -- очень напоминает функции
                              # языков программирования с их изолированным пространством имён

В данном случае, однако, не стоит слишком сильно увлекаться ассоциацией функций и subshell &ndash; это именно что другой процесс и единственный способ получить из него данные &ndash; захватить то что он выдаёт.

In [ ]:
%%bash
# Этот код работает

a=1

set  # а это выводит список всех переменных и функций в сессии

И классика жанра: циклы

In [ ]:
%%bash
# Этот код работает

for i in {1..10};
do
    echo "$i"
done

In [ ]:
%%bash
# Этот код работает

for i in $(ls -la);  # каждый пробел bash воспринимает как разделитель элементов массива
do
    echo "$i"
done

#### Проверка корректности

Посчитайте суммарное количество файлов и папок в папках Documents и Downloads.

### Скрипты и переменные окружения

Как на любом уважающем себя языке, на bash можно писать скрипты:

In [ ]:
%%bash
# Этот код работает

cat > my_script.sh <<EOF  # создадим скрипт
#!/bin/bash
# это нужно для того, чтобы редакторы кода понимали для какого языка делаеть подсветку синтаксиса

echo "Hello!"
EOF

chmod +x my_script.sh     # вот тут и понадобился chmod: по умолчанию файл имеет права rw и нужно добавить права x

./my_script.sh            # а вот так скрипт выполняется

Но есть нюанс &ndash; переменные в скрипты не передаются:

In [ ]:
%%bash
# Этот код работает

a=5

cat > my_script.sh <<"EOF"
#!/bin/bash

echo "a=$a"
EOF

chmod +x my_script.sh

./my_script.sh

Это связано с тем, что скрипт выполняется в новой сессии shell, не являющейся дочерней для текущей, то есть никакие данные из текущей сессии туда не копируются.

Раз обычные переменные не передаются в скрипты, должы быть какие-то другие способы передать данные. И способа целых два, первый &ndash; это environment variables (переменные окружения), которые соответствуют глобальным переменным в других языках программирования:

In [ ]:
%%bash
# Этот код работает

export A=1  # создаём переменную окружения

cat > my_script.sh <<"EOF"
#!/bin/bash

echo "A=$A"

A=5

export B=6
EOF

chmod +x my_script.sh

./my_script.sh

echo "A=$A, B=$B"  # а вот назад из скрипта environment variables не передаются -- передача идёт только по
                   #                                                                    нисходящей вызовов скриптов

In [ ]:
%%bash
# Этот код работает

env  # позволяет посмотреть все имеющиеся переменные окружения

Переменные окружения часто используются для настройки программ и передачи данных между процессами, например, чтобы указать backend серверу ссылку из под которой виден frontend, чтобы правильно перенаправлять запрос и рендерить файлы или по какому адресу находится база данных и какой от неё пароль. Заходя в систему у вас уже есть немало переменных окружения. Например, переменная `HOME` указывает на домашнюю папку, которая открывается, когда вы пишете `cd ~`, а переменная `PATH` хранит пути по которым bash ищет исполняемые скрипты. Чтобы не писать каждый раз как мы это делали `./my_script.sh`, а если точнее `<path to script>/my_script.sh`, мы можем добавить папку в которой лежит наш скрипт в `PATH` и скрипт будет вызываться просто так `my_script.sh`. Кстати, `find` или `env` &ndash; тоже скрипты и они лежат в папке `/bin` (~ binary &ndash; нетекстовое/скомпилированное под систему).

К слову, есть вариант выполнить скрипт не в новой сессии shell, а в текущей, для этого используют команду `source`: `source my_script.sh` или что то же самое `. my_script.sh`.

In [ ]:
%%bash
# Этот код работает

export PATH="$PATH:$(pwd)"                         # чтобы не стереть весь PATH -- добавляем наш путь в конец

my_script.sh

ls /bin | grep -e "^env$" -e "^find$" -e "^grep$"  # все эти скрипты лежат в /bin

Однако, есть бяда: наши envs (так сокращают environment variables) исчезают, стоит нам перезапустить сессию shell, но иногда, хочется, чтобы они были вечны. Тогда на помощь приходит файлик `~/.bash_profile`, который позволяет выполнять команды при запуске командной строки и может дать долголетие нашим энвам.

In [ ]:
%%bash
# Этот код работает

echo "export MY_ENV_VAR=\"Hello world!\"" >> ~/.bash_profile

In [ ]:
%%bash
# Этот код работает

bash -c 'echo $MY_ENV_VAR'

#### Проверка корректности

Создайте скрипт, читающий пользователей из файла и генерирующий случайный пароль для каждого в списке. Результат должен записываться в отдельный файл в формате `имя пользователя: пароль`. Сделайте так, чтобы его можно было выполнять не указывая путь до него. Может пригодится команда base64, которая принимает текст или байты на вход и выдаёт строку из читаемых символов (пример использования: `echo "Hello World!" | base64`).

### Аргументы и операторы

Возвращаясь к скриптам, второй способ передачи данных в скрипт &ndash; аргументы скрипта. С некоторыми вы уже знакомы, например, флаг `-a` в команде `ls -a` &ndash; это аргумент, как и `~` в команде `cd ~`. Соответственно `ls` и `cd` &ndash; такие же скрипты, как и те, что пишем мы с вами. 

In [ ]:
%%bash
# Этот код работает

cat > my_script.sh <<"EOF"
#!/bin/bash

echo "Script path: $0"

echo "All arguments: $@"

echo "First argument: $1"

echo "Second argument: $2"

echo "last argument: ${@: -1}"   # $@ -- массив в bash, соответственно ${array: index} позволяет брать элемент 
                                 #                                                    массива array с номером index

echo "Tenth argument: ${10}"     # не существует => пустое место

SCRIPT=$(readlink -f "$0")       # по относительному пути скрипта, возвращаемому в $0 возвращает полный путь скрипта

SCRIPTPATH=$(dirname "$SCRIPT")  # выдаёт папку в которой лежит указанный файл

echo $SCRIPT
echo $SCRIPTPATH
EOF

chmod +x my_script.sh

mkdir test && cd test

../my_script.sh -l test --help   # вызываем скрипт с параметрами

cd .. && rm -r test

Как видим, флаг &ndash; это тот же параметр, ничего загадочного в нём нет. Но тогда как команды типа ls разбираются с флагами? 

**Эта часть необязательна**

In [ ]:
%%bash
# Этот код работает

cat > my_script.sh <<"EOF"
#!/bin/bash

options=":f:ed"             # первое двоеточие нужно, чтобы скрипт выбирал вариант \? если указана какая-либо
                            #                                                                     неуказанная опция
                            # далее идут буквы флагов (опции), если после буквы стоит двоеточие -- этот флаг ожидает
                            #                                                                              аргумента

while getopts $options opt  # это цикл while, самый обычный, а команда getopts перебирает опции на каждой итерации,
do                          #                                                    записывая текущую в переменную opt

        case $opt in        # switch: если переменная opt равна какой-то из перечисленных букв 
                            # (указан соответствующий флаг -f, -e или -d), то выполняется одна из следующих строчек
                            
                f) echo "-f flag chosen"; echo "-f flag argument: $OPTARG";;    # OPTARG -- аргумент флага
                                                            # ;; - необходимо, чтобы найдя первое совпадение флага
                                                            # прекращалась проверка дальнейших пунктов и while
                                                            # уходил на следующую итерацию.
                
                e) echo "-e flag chosen" ;;                 # флаг не ожидающий аргумента
                d) echo "-d flag chosen" ;;
                \? ) echo "Unknown option: -$OPTARG" >&2;;  # вызывается, если указана 
                                                            #                 неизвестная опция
                                                                                
                :  ) echo "Missing option argument for -$OPTARG" >&2;;  # вызывается, если указан флаг,
                                                                        # ожидающий аргумента, но аргумент
                                                                        #                        не передан
                                                                                
                *  ) echo "Default option";;                            # вызывается, если ни один из выше- 
                                                                        # перечисленных вариантов не сработал
                                                                          
        esac
        
done


EOF

chmod +x my_script.sh

./my_script.sh -f "./authData.yaml.zip" -e -a

Иногда необходимо принимать решения, тогда на помощь приходит if-else:

**А эта обязательна**

In [ ]:
%%bash
# Этот код работает

cat > my_script.sh <<"EOF"                   # скрипт провряет расширение файла
#!/bin/bash


if [ ! -z "$(echo "$1" | grep ".yaml")" ]    # тестовое условие заключено в квадратные скобки, "!" означает not, "-z" 
                                             # тестирует на то, не пустая ли строчка
then
    echo "entered yaml file"
elif [ ! -z "$(echo "$1" | grep ".json")" ]  # grep фильтрует расширение файла
then
    echo "entered json file"
else
    echo "no file entered"
    exit 1;
fi

EOF

chmod +x my_script.sh

./my_script.sh "./authData.yaml"

## Задачи

1. Напишите скрипт, заменяющий расширение в имени файла на указанное. В качестве параметров после флагов будет передаваться исходное имя файла и расширение на которое нужно заменить.

2. Напишите скрипт, который имплементирует функцию поиска файлового проводника, в зависимости от флага ищет или по именам файлов или по их содержанию рекурсивно в указанной папке.

3. Напишите скрипт, который в зависимости от флага зашифровывает/расшифровывает файл паролем (подсказка: используйте OpenSSL). 